# importação de backups

In [2]:
import pyodbc as PDB  # Drive para conexão com banco de dados
import pandas  as pd
import sys
import os
import traceback
import json

In [3]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

In [4]:
print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")

Azure Blob Storage v12.8.0 - Python quickstart sample


## Importa o arquivo com as configurações


In [9]:
datasource = pd.read_excel(r'D:\Ondrive\OneDrive - Implanta Informatica\Migração arquivos Anexos\GeracaoContainnersStorageAzure.xlsx',sheet_name='registros')

# Chave dos containers
improblob01

In [25]:
for index , row in datasource[0:1].iterrows():
    database = row["banco"]
    container =row["Container"]
    storageAccount =  row["StorageAccount"]
    connect_str_to_storage = row["ConnectionStringAzureStorageArquivosAnexos"]

    blob_service_client = BlobServiceClient.from_connection_string(connect_str_to_storage)

    existContainer = blob_service_client.get_container_client(container).exists()

    
    if(existContainer ==False):
        blob_service_client.create_container(container)
        print('database:',database,' storageAccount:',storageAccount,' Container Criado',container)
    else:
         print('database:',database,' storageAccount:',storageAccount,' Container já existe',container)


    


database: cau-al.implanta.net.br  storageAccount: improblob01  Container já existe cau-al
